# Optimización con Vectorización en Python
En este notebook exploraremos cómo aprovechar la vectorización en Python usando NumPy y PyTorch, así como los conceptos clave de SIMD, BLAS y la optimización de inferencia.


## Conceptos Clave de Vectorización y Eficiencia Computacional

La vectorización no solo simplifica el código, sino que proporciona **mejoras sustanciales en rendimiento** al aprovechar:
- **Instrucciones SIMD**: Modernas CPUs incluyen extensiones (SSE, AVX, NEON) que ejecutan la misma operación sobre múltiples datos en un solo ciclo, procesando en paralelo vectores de 4, 8 o más valores.
- **Operaciones en C optimizado**: Las funciones universales de NumPy (ufuncs) están escritas en C, evitando el overhead del intérprete Python en cada iteración.
- **Localidad de memoria**: Al trabajar con arrays contiguos, se maximizan los “cache hits” de L1/L2/L3, reduciendo dramáticamente los accesos a memoria RAM.
- **BLAS/MKL/OpenBLAS multihilo**: Para álgebra lineal pesada (`np.dot`, `np.linalg.inv`, multiplicaciones matriciales), estas bibliotecas distribuyen el cómputo entre varios núcleos, equilibrando carga y minimizando tiempos de espera.
- **Broadcasting**: Permite operar sin crear copias innecesarias de arrays, reusando la misma memoria y evitando loops Python implícitos.
- **Menos GIL**: Al delegar todo el trabajo en C (o en GPU para PyTorch/TensorFlow), se liberan cuellos de botella asociados al Global Interpreter Lock de Python.
- **GPU y Tensor Cores**: En Deep Learning, frameworks vectorizados extienden estos principios al hardware de GPU, usando miles de núcleos paralelos y Tensor Cores para acelerar multiplicaciones de matrices a precisión reducida (FP16/TF32).

> **Reflexión**: Al vectorizar, transformas tu código de una serie de bucles Python en una composición de operaciones de alto nivel ejecutadas directamente en hardware especializado, logrando **ganancias de 10× a 100×** en muchas tareas numéricas.



## Detalle del Pipeline de Optimización

1. **Identificación de cuellos de botella**: Usa `%timeit`, `cProfile` o `line_profiler` para detectar loops costosos.
2. **Reemplazo por ufuncs**: Sustituye cada iteración por llamadas a `np.add`, `np.multiply`, `np.dot`, etc.
3. **Validación de shapes**: Al usar broadcasting, verifica que las dimensiones sean compatibles para evitar bugs silenciosos.
4. **Ajuste de dtype**: Selecciona tipos de datos más ligeros (`float32` vs `float64`) para reducir carga de memoria y acelerar cálculos.
5. **Test de rendimiento**: Mide antes y después de cada cambio para asegurar que la vectorización aporta mejoras reales.


## Implementación del Código Completo
A continuación presentamos el código con docstrings y comentarios explicativos, demostrando generación de datos, entrenamiento y una inferencia vectorizada.

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import time

def generar_datos_sinteticos(num_muestras=5000):
    """
    Genera un conjunto de datos sintético para clasificación binaria.
    Cada muestra es un vector de 10 características muestreado de una distribución
    normal estándar. La etiqueta es 1 si la suma de las características es positiva,
    y 0 en caso contrario.

    Args:
        num_muestras (int): Número de ejemplos a generar.

    Returns:
        X (np.ndarray): Matriz de características de forma (num_muestras, 10).
        y (np.ndarray): Vector de etiquetas binarias de longitud num_muestras.
    """
    X = np.random.randn(num_muestras, 10)
    y = (np.sum(X, axis=1) > 0).astype(int)
    return X, y

class RedNeuronalSimple(nn.Module):
    """
    Una red neuronal de dos capas para clasificación binaria.
    Arquitectura: Linear -> ReLU -> Linear.

    Args:
        dim_entrada (int): Características de entrada.
        dim_oculta (int): Neuronas capa oculta.
        dim_salida (int): Clases de salida.
    """
    def __init__(self, dim_entrada=10, dim_oculta=32, dim_salida=2):
        super().__init__()
        self.modelo = nn.Sequential(
            nn.Linear(dim_entrada, dim_oculta),
            nn.ReLU(),
            nn.Linear(dim_oculta, dim_salida)
        )

    def forward(self, x):
        return self.modelo(x)

def entrenar_modelo(modelo, X_train, y_train, epochs=5, batch_size=128):
    """
    Entrena el modelo con Adam y CrossEntropyLoss. Usa DataLoader con num_workers=2.
    """
    optimizer = optim.Adam(modelo.parameters(), lr=1e-3)
    criterio = nn.CrossEntropyLoss()
    modelo.train()

    tensor_X = torch.tensor(X_train, dtype=torch.float32)
    tensor_y = torch.tensor(y_train, dtype=torch.long)
    dataset = torch.utils.data.TensorDataset(tensor_X, tensor_y)
    loader  = torch.utils.data.DataLoader(
        dataset, batch_size=batch_size, shuffle=True, num_workers=2
    )

    for epoch in range(epochs):
        perdida_acum = 0.0
        for bx, by in loader:
            optimizer.zero_grad()
            salidas = modelo(bx)
            loss = criterio(salidas, by)
            loss.backward()
            optimizer.step()
            perdida_acum += loss.item()
        print(f"Epoch {epoch+1}/{epochs}, Pérdida Promedio: {perdida_acum/len(loader):.4f}")

def inferencia_vectorizada(modelo, X_data, batch_size=None, device='cpu'):
    """
    Realiza inferencia vectorizada: procesa todo el batch en C/CUDA.
    Args:
        modelo (nn.Module): Red entrenada.
        X_data (np.ndarray): Datos de prueba.
        batch_size (int|None): Divide en batches si es necesario.
        device (str): 'cpu' o 'cuda'.
    Returns:
        np.ndarray: Predicciones.
    """
    modelo.to(device).eval()
    with torch.no_grad():
        tensor_X = torch.tensor(X_data, dtype=torch.float32, device=device)
        if batch_size:
            preds = []
            for i in range(0, len(tensor_X), batch_size):
                batch = tensor_X[i:i+batch_size]
                out = modelo(batch)
                preds.append(out.argmax(dim=1).cpu())
            return torch.cat(preds).numpy()
        else:
            out = modelo(tensor_X)
            return out.argmax(dim=1).cpu().numpy()

def main():
    """
    1. Genera datos.
    2. Divide en entrenamiento/test.
    3. Entrena red.
    4. Infere vectorizado.
    5. Mide tiempos y precisión.
    """
    X, y = generar_datos_sinteticos(5000)
    split = int(0.8 * len(X))
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]

    modelo = RedNeuronalSimple()

    print("Entrenando...")
    t0 = time.time()
    entrenar_modelo(modelo, X_train, y_train, epochs=5, batch_size=128)
    print(f"→ Entrenamiento en {time.time() - t0:.2f}s\n")

    print("Inferencia vectorizada...")
    t1 = time.time()
    pred = inferencia_vectorizada(modelo, X_test)
    print(f"→ Inferencia en {time.time() - t1:.2f}s")

    precision = (pred == y_test).mean() * 100
    print(f"Precisión en Test: {precision:.2f}%")

if __name__ == '__main__':
    main()
